# Storing Secrets for your labs

You can use the Microsoft.PowerShell.SecretManagement module introduced here to store and retrieve secrets with PowerShell.

Installation
You install it from the PowerShell Gallery using

In [ ]:
Install-Module Microsoft.PowerShell.SecretManagement -AllowPrerelease

You need to use the `-PreRelease` flag at the moment as it is in preview. 

## Local Vault

It will come with a built in local key vault for you to use. You can see this with `Get-SecretVault`

In [1]:
Get-SecretVault

Name              ModuleName ImplementingType


----              ---------- ----------------


BuiltInLocalVault            


## Store your secrets

This is a great place to store your secrets for your demos. You can add your secrets using a couple of methods

- It's just your lab, just type them in to `Get-Secret`

In [2]:
Set-Secret -Name BeardLabSaAccount -Secret 'BeardsAreAwesome'